# Módulo 2: Scraping con Selenium
## LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

**¡Empecemos!**
Utilicemos lo aprendido hasta ahora para lograr el objetivo propuesto

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.latam.com/es_ar/apps/personas/booking?fecha1_dia=18&fecha1_anomes=2019-12&auAvailability=1&ida_vuelta=ida&vuelos_origen=Buenos%20Aires&from_city1=BUE&vuelos_destino=Madrid&to_city1=MAD&flex=1&vuelos_fecha_salida_ddmmaaaa=18/12/2019&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo=#/'

In [3]:
r = requests.get(url)

In [4]:
r.status_code

200

In [5]:
s = BeautifulSoup(r.text, 'lxml')

In [6]:
print(s.prettify())

<!DOCTYPE html>
<html lang="es">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <title>
   Selecciona tus vuelos | LATAM Airlines
  </title>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="https://s.latamstatic.com/static/latam/images/favicon/LATAM/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <meta content="personas_bs3_latam_mu" name="layout"/>
  <meta content="LAN.com" name="description"/>
  <meta content="LAN.com" name="keywords"/>
  <meta content="B90D7B166BC8D2088504237E75BD91D8" name="sessionId" scheme="tracker"/>
  <meta content="latam" name="matchedScenario" scheme="tracker"/>
  <meta content="personas_bs3_latam_mu" name="template" scheme="tracker"/>
  <meta content="null" name="userDataCookieRawData" scheme="tracker"/>
  <meta content="3.0.53" name="appVersion" scheme="tracker"/>
  <style>
   .async-hide{opacity:0!important}
  

Vemos que la respuesta de la página no contiene la información que buscamos, ya que la misma aparece recién después de ejecutar el código JavaSCript que está en la respuesta.

## Selenium

Selenium es una herramienta que nos permitirá controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto necesitamos el módulo `webdriver`.

In [7]:
from selenium import webdriver

Paso 1: instanciar un **driver** del navegador

In [20]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='../chromedriver', options=options)

Paso 2: hacer que el navegador cargue la página web.

In [21]:
driver.get(url)

Paso 3: extraer la información de la página

In [22]:
vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
vuelos

[<selenium.webdriver.remote.webelement.WebElement (session="dbf484ccc25d5c7c257eec0196615cc6", element="fc9d09fa-89d7-47a0-98cb-16d88d85be10")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dbf484ccc25d5c7c257eec0196615cc6", element="009bb343-534b-40f7-9382-3e68e50cfab8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dbf484ccc25d5c7c257eec0196615cc6", element="228d7855-6692-4791-8f8e-d021a4d95bb2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dbf484ccc25d5c7c257eec0196615cc6", element="44ef90ae-c81d-4cfa-a178-739997997af8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dbf484ccc25d5c7c257eec0196615cc6", element="0ed603e0-1014-480d-a4eb-f77be31f3824")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dbf484ccc25d5c7c257eec0196615cc6", element="1dd4c17f-8411-4fc0-aa2a-3b3f6f583076")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dbf484ccc25d5c7c257eec0196615cc6", element="876947d0-4320-4d78-8d7f-72

In [23]:
vuelo = vuelos[0]

In [24]:
vuelo

<selenium.webdriver.remote.webelement.WebElement (session="dbf484ccc25d5c7c257eec0196615cc6", element="fc9d09fa-89d7-47a0-98cb-16d88d85be10")>

In [25]:
# Hora de salida
vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')

'19:20'

In [26]:
# Hora de llegada
vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')

'13:10'

In [27]:
# Duración del vuelo
vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

'PT13H50M'

In [28]:
boton_escalas = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')

In [30]:
boton_escalas.click()

In [31]:
segmentos = vuelo.find_elements_by_xpath('//div[@class="segments-graph"]/div[@class="segments-graph-segment"]')
segmentos

[<selenium.webdriver.remote.webelement.WebElement (session="dbf484ccc25d5c7c257eec0196615cc6", element="eb9786e9-73bd-404d-b5b5-31b5b834ec6a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dbf484ccc25d5c7c257eec0196615cc6", element="8b617e09-1e04-4ab5-8820-663796cefcb1")>]

In [32]:
escalas = len(segmentos) - 1 

In [33]:
escalas

1

Paso 4: cerrar el navegador

In [18]:
driver.close()